<a href="https://colab.research.google.com/github/aekaba/RestoranReviewsNLP/blob/main/NLPRestoranReviewAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DDİ Ödev 2 Soru 2
## Derlem Özellikleri
Seçtiğim derlem http://metashare.ilsp.gr:8080/repository/browse/semeval-2016-absa-restaurant-reviews-turkish-train-data-subtask-1/ff5dad70676311e5bf9c842b2b6a04d71fa7fa3ba4504a228dafe0c24560585b/
içinde 320 cümle bulunduruyo, bunlar 2016 yılının restoran geri dönüşlerini içeren cümleler ve 1232 kelime bulunan bir derlem oluşturulmuştur. Veriler bir xml dosyası içerisinde tutulmaktadır.
## Çalışmanın Hedefi
Çalışma, hem hedef alan hemde duygu analizi içeren bir model geliştirmek. Böylelikle geliştirilen model, restoranların hangi alanlarının iyi olduğunu hangi alanlarının kötü olduğunu belirtebilmektedir.
## Yöntem
Çalışma Google Colab üzerinde geliştirilmiştir. Bir python not defteri oluşturularak her adım üzerinde açıklama yapılarak anlatılmıştır. Bir python kütüphanesi olan nltk kullanarak doğal dil işleme adımları yapılmıştır. Model Naive bayes makine öğrenme modeli kullanılarak geliştirilmiştir. Veri okuma ve ön işleme aşamaları pandas kütüphanesi ile gerçekleştirilmiştir.
1. Veri Okuma ve Data Frame oluşturma  
  python xml kütüphanesi ile google colab'a yüklediğimiz xml dosyasını açıp, satır satır okuyoruz. İçindeki hali hazırda etiketlenmiş verileri bir dataframe e aktarıyoruz.
  Verilerimiz Text - Target - Category - Polarity olmak üzere 4 sütuna ayrılmıştır.  
  Text: Cümlenin kendisi.   
  Target: Yorumu ilgilendiren bölüm.  
  Category: Metinde bahsedilen hedefin nasıl daha geniş bir sınıfa dahil olduğunu belirtir.  
  Polarity: Metnin duygusunu belirten kısım.  
  bir cümlenin birden fazla görüşü olabilir.




In [9]:
import xml.etree.ElementTree as ET
import pandas as pd

# XML dosyasını okuma
tree = ET.parse('/content/reviews.xml')  # Derlem Dosyamız
root = tree.getroot()

data = []

for review in root.findall("Review"):
    for sentence in review.find("sentences").findall("sentence"):
        if "OutOfScope" not in sentence.attrib:
            text = sentence.find("text").text
            opinions = sentence.find("Opinions")
            if opinions is not None:
                for opinion in opinions.findall("Opinion"):
                    target = opinion.attrib["target"]
                    category = opinion.attrib["category"]
                    polarity = opinion.attrib["polarity"]
                    data.append({"text": text, "target": target, "category": category, "polarity": polarity})

# DataFrame oluşturma
df = pd.DataFrame(data)
print(df.head())

                                                text     target  \
0            Manzara sahane evet ama servis rezalet.     servis   
1            Manzara sahane evet ama servis rezalet.    Manzara   
2  Soguk su isteyince, soguk yok, butun sulari di...    mezenin   
3  Soguk su isteyince, soguk yok, butun sulari di...     garson   
4  Yemekler iyi hos, lezzetler iyi ama heyecan ve...  lezzetler   

             category  polarity  
0     SERVICE#GENERAL  negative  
1    AMBIENCE#GENERAL  positive  
2  FOOD#STYLE_OPTIONS  negative  
3     SERVICE#GENERAL  negative  
4        FOOD#QUALITY  positive  


2. Veri Ön İşleme  
  Burada oluşturulan dataframe içinde bulunan metin kısmının temizlenmesi gerekmektedir. Noktalama işaretleri kaldırlıcak ve tüm metinler küçük harflere dönüştürülecektir. Sonrasında temizlenmiş veri yeni bir stün olarak eklenecektir. Sonrasında algoritmamızın alnayabilmesi için kategori ve hedef kısmını sayısal verilere çevirmemiz gerekecektir. Her biri sayısal olarak etiketlenip dataframe de yeni bir sütun a atanacaktır.


In [21]:
from sklearn.preprocessing import LabelEncoder
import re
import nltk

nltk.download('stopwords')
turkish_stop_words = stopwords.words('turkish')
# Metin temizleme
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldır
    text = text.lower()  # Küçük harfe dönüştür
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

# Etiketleme
le_category = LabelEncoder()
le_polarity = LabelEncoder()

df['category_label'] = le_category.fit_transform(df['category'])
df['polarity_label'] = le_polarity.fit_transform(df['polarity'])

print("Kategori Etiketleri:", dict(zip(le_category.classes_, le_category.transform(le_category.classes_))))
print("Duygu Etiketleri:", dict(zip(le_polarity.classes_, le_polarity.transform(le_polarity.classes_))))

Kategori Etiketleri: {'AMBIENCE#GENERAL': 0, 'DRINKS#PRICES': 1, 'DRINKS#QUALITY': 2, 'DRINKS#STYLE_OPTIONS': 3, 'FOOD#PRICES': 4, 'FOOD#QUALITY': 5, 'FOOD#STYLE_OPTIONS': 6, 'LOCATION#GENERAL': 7, 'RESTAURANT#GENERAL': 8, 'RESTAURANT#MISCELLANEOUS': 9, 'RESTAURANT#PRICES': 10, 'SERVICE#GENERAL': 11}
Duygu Etiketleri: {'negative': 0, 'neutral': 1, 'positive': 2}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


3. Öznitelik Çıkarma.  
  Burada metinleri, makine öğrenmesi algoritmalarının anlayabileceği bir şekile yani sayısal formata dönüştürecez. Bunun için TF-IDF tekniği kullanılmıştır. Bir kelimenin metin içinde ne sıklıkla geçtiğini ve bu kelimenin ne kadar ayırt edici olduğunu ölçen bir yöntemdir.

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörleştirme
vectorizer = TfidfVectorizer(max_features=50, stop_words=turkish_stop_words)
X = vectorizer.fit_transform(df['cleaned_text']).toarray()

# Hedef alan ve duygu etiketleri
y_category = df['category_label']
y_polarity = df['polarity_label']



4. Model Eğitimi.  
  Hedef alanları ve duygu analizi için naive bayes modelini eğitelim

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Eğitim ve test bölme
X_train, X_test, y_category_train, y_category_test = train_test_split(X, y_category, test_size=0.2, random_state=42)
X_train, X_test, y_polarity_train, y_polarity_test = train_test_split(X, y_polarity, test_size=0.2, random_state=42)

# Hedef alan modeli
category_model = MultinomialNB()
category_model.fit(X_train, y_category_train)
y_category_pred = category_model.predict(X_test)

# Duygu modeli
polarity_model = MultinomialNB()
polarity_model.fit(X_train, y_polarity_train)
y_polarity_pred = polarity_model.predict(X_test)

# Sonuçları değerlendirme
print("Hedef Alan Modeli:")
print(classification_report(y_category_test, y_category_pred, target_names=le_category.classes_, zero_division=0))

print("Duygu Modeli:")
print(classification_report(y_polarity_test, y_polarity_pred, target_names=le_polarity.classes_, zero_division=0))


Hedef Alan Modeli:
                          precision    recall  f1-score   support

        AMBIENCE#GENERAL       0.45      0.25      0.32        57
           DRINKS#PRICES       0.00      0.00      0.00         3
          DRINKS#QUALITY       0.00      0.00      0.00        15
    DRINKS#STYLE_OPTIONS       0.00      0.00      0.00         7
             FOOD#PRICES       1.00      0.33      0.50         3
            FOOD#QUALITY       0.37      0.77      0.50        93
      FOOD#STYLE_OPTIONS       0.00      0.00      0.00        23
        LOCATION#GENERAL       0.00      0.00      0.00         7
      RESTAURANT#GENERAL       0.23      0.28      0.25        43
RESTAURANT#MISCELLANEOUS       0.00      0.00      0.00         1
       RESTAURANT#PRICES       0.25      0.14      0.18         7
         SERVICE#GENERAL       0.56      0.29      0.38        48

                accuracy                           0.37       307
               macro avg       0.24      0.17      0.18

5. Modeli Kullanalım.  
  

In [71]:
# Yeni metin için tahmin
new_text = ["Fiyatlar çok fazla"]
new_text_cleaned = [clean_text(t) for t in new_text]
new_X = vectorizer.transform(new_text_cleaned).toarray()

# Tahminler
category_prediction = le_category.inverse_transform(category_model.predict(new_X))
polarity_prediction = le_polarity.inverse_transform(polarity_model.predict(new_X))

print("Hedef Alan Tahmini:", category_prediction)
print("Duygu Tahmini:", polarity_prediction)


Hedef Alan Tahmini: ['RESTAURANT#PRICES']
Duygu Tahmini: ['negative']


## Performans Analizi
Hedef alan modeli genel başarısı: %40. Hedefleri tespit etmede ortalama bir başarı göstermekte fakat bazı kategoriler için başarı olanı oldukça düşük. %77 ile recall değeri ile FOOD#QUALITY en doğru tahminde bulunan kategori olmuştur. Diğer kategorilerin değerleri oldukça düşüktür.  
Duygu modeli genel başarısı: %63. Model duygu alanizi açısından daha iyi sonuç vermiştir. Pozitif ne negatif değerler kabul edilebilir bir başarı göstermiştir. Notr kısım ise çok düşük performans göstermektedir.  
Bunların başlıca sebebi veri setinin yetersiz ve dengesiz olmasından kaynaklanmıştır. Daha fazla etiketlenmiş veri bularak yada, az olan kategorileri çıkarak yada naive bayes den daha karmaşık algoritmalar kullanarak (Random Forest, Logistic Regression, vb.) daha başarılı bir model eğitebiliriz.
